In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import requests
import pandas as pd
import json
from citipy import citipy
import random

# Import API key
from config import api_key

In [2]:
# Import City of Melbourne Pedestrian Counting System Sensor Locations as a Pandas DataFrame
sensors_df = pd.read_csv("../Resource/PCS_Locations.csv")
sensors_df

,sensor_id,sensor_description,sensor_name,installation_date,status,note,direction_1,direction_2,latitude,longitude,location
0,55,Elizabeth St-La Trobe St (East),Eli380_T,2018/07/19,A,NaN,North,South,-37.809889,144.961343,"(-37.80988941, 144.96134331)"
1,45,Little Collins St-Swanston St (East),Swa148_T,2017/06/29,A,NaN,North,South,-37.814141,144.966094,"(-37.81414074, 144.9660938)"
2,61,RMIT Building 14,RMIT14_T,2019/06/28,A,NaN,North,South,-37.807675,144.963091,"(-37.80767455, 144.96309115)"
3,7,Birrarung Marr,Fed_T,2014/12/17,A,NaN,East,West,-37.818629,144.971694,"(-37.81862929, 144.97169395)"
4,72,Flinders St- ACMI,ACMI_T,2020/11/30,A,NaN,East,West,-37.817263,144.968728,"(-37.81726337, 144.96872809)"
...,...,...,...,...,...,...,...,...,...,...,...
69,51,QVM-Franklin St (North),Fra118_T,2017/11/30,A,NaN,West,East,-37.808418,144.959063,"(-37.80841814, 144.95906317)"
70,10,Victoria Point,BouHbr_T,2009/04/23,A,NaN,West,East,-37.818765,144.947105,"(-37.81876473, 144.94710545)"
71,43,Monash Rd-Swanston St (West),UM2_T,2015/04/15,A,NaN,South,North,-37.798445,144.964118,"(-37.79844525, 144.96411782)"
72,73,Bourke St - Spencer St (South),Bou655_T,2020/10/02,A,NaN,West,East,-37.816957,144.954154,"(-37.81695684, 144.95415373)"


In [3]:
sensors_df["sensor_description"].unique()

array(['Elizabeth St-La Trobe St (East)',
       'Little Collins St-Swanston St (East)', 'RMIT Building 14',
       'Birrarung Marr', 'Flinders St- ACMI', 'La Trobe St (North)',
       '231 Bourke St', 'Sandridge Bridge',
       'Flinders La-Swanston St (West)',
       'Melbourne Convention Exhibition Centre',
       'Flinders La - Swanston St (West) Temporary',
       'Royal Pde - Grattan St', 'Alfred Place', 'Collins Place (South)',
       'Swanston St - City Square', 'State Library - New',
       'State Library', 'Flinders Ln -Degraves St (South)',
       'Melbourne Central-Elizabeth St (East)', 'Lincoln-Swanston (West)',
       'Flinders Ln -Degraves St (North)',
       'Flinders Ln -Degraves St (Crossing)', 'Melbourne Central',
       'St Kilda Rd-Alexandra Gardens', 'Errol Street (East)',
       'Westwood Place', 'Bourke St-Russell St (West)',
       'Spencer St-Collins St (South)', 'Flinders St-Spring St (West)',
       'Flagstaff Station', 'New Quay', 'Collins Street (North)',


In [4]:
# Only use the following 3 sensors:
sensors_reduced = sensors_df.copy()
sensors_reduced = sensors_reduced.loc[(sensors_reduced["sensor_description"] == "Bourke Street Mall (North)") | 
                                        (sensors_reduced["sensor_description"] == "QV Market-Elizabeth St (West)") |
                                        (sensors_reduced["sensor_description"] == "Flinders St-Spark La")
                                         ,:]
sensors_reduced[["sensor_id","sensor_description"]]

,sensor_id,sensor_description
46,1,Bourke Street Mall (North)
49,34,Flinders St-Spark La
61,26,QV Market-Elizabeth St (West)


In [5]:
# Base URL required for API call
url = "https://developers.zomato.com/api/v2.1/search?"

# Parameters to run through Query URL
# Note, Zomato API returns 20 search items at a time, therefore 
# Multiple calls required at "start" intervals of 20
start = np.arange(0,40,20)
count = 20
city = 259
radius = 50
b = 0

In [6]:
# Latitude and Longitudes of Sensor Locations:
lat = list(sensors_reduced["latitude"])
lon = list(sensors_reduced["longitude"])

In [7]:
# Empty list to hold all API calls
response_data = []

In [8]:
# Loop through Zomato API
for a in range(len(start)):
    for b in range(len(lat)):
        query_url = f"{url}entity_id={city}&entity_type=metro&lat={lat[b]}&lon={lon[b]}&radius={radius}&sort=real_distance&apikey={api_key}&start={start[a]}&count={count}"
        response_data.append(requests.get(query_url).json())
        print(start[a], lat[b], lon[b])
    b += 1

0 -37.813494399999996 144.96515324
0 -37.81537985 144.97415049999998
0 -37.80869542 144.96049404
20 -37.813494399999996 144.96515324
20 -37.81537985 144.97415049999998
20 -37.80869542 144.96049404


In [18]:
# confirm expected amount of data has been retrieved
# If six, then the Zomato API has been retrieved successfully twice per location
len(response_data)

6

In [19]:
# Empty list to hold all restaurant names, aggregate rating, hours of operation, type of eatery, price 
restr_names = []
rating = []
cuisine_type = []
price_range = []
restr_lat = []
restr_lon = []
sensor_loc = []
sensor_id = []
c = 0
d = 0

# Populate lists with JSON data from Zomato 
for c in range(len(response_data)):     
    # If up to the fourth loop, restart "d" to assign the first location to the data. 
    if c == 3:
        d = 0
    else:
        d = d
    for x in range(count):
        restr_names.append(response_data[c]["restaurants"][x]["restaurant"]["name"])
        rating.append(response_data[c]["restaurants"][x]["restaurant"]["user_rating"]["aggregate_rating"])
        cuisine_type.append(response_data[c]["restaurants"][x]["restaurant"]["cuisines"])
        price_range.append(response_data[c]["restaurants"][x]["restaurant"]["price_range"])
        restr_lat.append(response_data[c]["restaurants"][x]["restaurant"]["location"]["latitude"])
        restr_lon.append(response_data[c]["restaurants"][x]["restaurant"]["location"]["longitude"])
        sensor_id.append(list(sensors_reduced["sensor_id"])[d])
        sensor_loc.append(list(sensors_reduced["sensor_description"])[d])
        print(c,x,d)
    if c < 3:
        d += 1 
    elif c >= 3:
        d += 1  
    c += 1

0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 5 0
0 6 0
0 7 0
0 8 0
0 9 0
0 10 0
0 11 0
0 12 0
0 13 0
0 14 0
0 15 0
0 16 0
0 17 0
0 18 0
0 19 0
1 0 1
1 1 1
1 2 1
1 3 1
1 4 1
1 5 1
1 6 1
1 7 1
1 8 1
1 9 1
1 10 1
1 11 1
1 12 1
1 13 1
1 14 1
1 15 1
1 16 1
1 17 1
1 18 1
1 19 1
2 0 2
2 1 2
2 2 2
2 3 2
2 4 2
2 5 2
2 6 2
2 7 2
2 8 2
2 9 2
2 10 2
2 11 2
2 12 2
2 13 2
2 14 2
2 15 2
2 16 2
2 17 2
2 18 2
2 19 2
3 0 0
3 1 0
3 2 0
3 3 0
3 4 0
3 5 0
3 6 0
3 7 0
3 8 0
3 9 0
3 10 0
3 11 0
3 12 0
3 13 0
3 14 0
3 15 0
3 16 0
3 17 0
3 18 0
3 19 0
4 0 1
4 1 1
4 2 1
4 3 1
4 4 1
4 5 1
4 6 1
4 7 1
4 8 1
4 9 1
4 10 1
4 11 1
4 12 1
4 13 1
4 14 1
4 15 1
4 16 1
4 17 1
4 18 1
4 19 1
5 0 2
5 1 2
5 2 2
5 3 2
5 4 2
5 5 2
5 6 2
5 7 2
5 8 2
5 9 2
5 10 2
5 11 2
5 12 2
5 13 2
5 14 2
5 15 2
5 16 2
5 17 2
5 18 2
5 19 2


In [20]:
# Make DataFrame from list data
zomato_df = pd.DataFrame({
    "Name": restr_names,
    "Aggregate Rating": rating,
    "Type": cuisine_type,
    "Price": price_range,
    "Latitude": restr_lat,
    "Longitude": restr_lon,
    "Sensor ID": sensor_id,
    "Sensor": sensor_loc
})

zomato_df.tail(10)

,Name,Aggregate Rating,Type,Price,Latitude,Longitude,Sensor ID,Sensor
110,HongKongBoi,3.9,"Asian, Desserts, Ice Cream",2,-37.8087495698,144.9593421444,26,QV Market-Elizabeth St (West)
111,Sushi Hub,3.3,"Sushi, Japanese",3,-37.8078213938,144.9600666761,26,QV Market-Elizabeth St (West)
112,B'Churrasco,4.4,"BBQ, Brazilian, Steak",4,-37.8090780311,144.9616310746,26,QV Market-Elizabeth St (West)
113,Gotcha,3.3,"Beverages, Bubble Tea, Tea",2,-37.8095346958,144.9611529708,26,QV Market-Elizabeth St (West)
114,Mary's,3.2,"Burger, Bar Food",3,-37.8089520000,144.9592780000,26,QV Market-Elizabeth St (West)
115,The Borek Bakehouse,4.4,"Bakery, Middle Eastern",1,-37.8077111987,144.9600053206,26,QV Market-Elizabeth St (West)
116,Chickcha,3.7,"Coffee and Tea, Cafe Food",2,-37.8090732631,144.9617665261,26,QV Market-Elizabeth St (West)
117,Damiao Panda Fan 大妙火锅 · 川菜,3.0,"Hot Pot, Sichuan, Chinese",3,-37.8096140000,144.9611990000,26,QV Market-Elizabeth St (West)
118,Mansae Korean BBQ & Bar,4.5,"Korean, Korean BBQ",4,-37.8095630386,144.9596073479,26,QV Market-Elizabeth St (West)
119,The Orient Hot Pot,3.8,"Chinese, Hot Pot, Sichuan",4,-37.8090205504,144.9618624151,26,QV Market-Elizabeth St (West)


In [21]:
# Drop duplicates from DataFrame
z_clean = zomato_df.drop_duplicates(subset="Name", keep=False).copy()
z_clean.count()

Name                111
Aggregate Rating    111
Type                111
Price               111
Latitude            111
Longitude           111
Sensor ID           111
Sensor              111
dtype: int64

In [22]:
# confirm datatypes of each column 
z_clean.dtypes

Name                object
Aggregate Rating    object
Type                object
Price                int64
Latitude            object
Longitude           object
Sensor ID            int64
Sensor              object
dtype: object

In [23]:
# Change column types of Rating, Latitude and Longitude from strings to numbers (float)
z_clean["Aggregate Rating"] = z_clean["Aggregate Rating"].astype(float)
z_clean["Latitude"] = z_clean["Latitude"].astype(float)
z_clean["Longitude"] = z_clean["Longitude"].astype(float)
z_clean

,Name,Aggregate Rating,Type,Price,Latitude,Longitude,Sensor ID,Sensor
0,Soul Cafe,3.0,"Coffee and Tea, Cafe Food",3,-37.813075,144.965281,1,Bourke Street Mall (North)
1,Song Tea,3.5,Bubble Tea,1,-37.813810,144.965600,1,Bourke Street Mall (North)
2,Sensory Lab - Bourke Street,3.4,"Coffee and Tea, Sandwich, Desserts",2,-37.813398,144.964499,1,Bourke Street Mall (North)
3,SamSam,4.0,Korean,3,-37.812885,144.965210,1,Bourke Street Mall (North)
4,Boost Juice,3.7,"Juices, Beverages",1,-37.813798,144.965849,1,Bourke Street Mall (North)
...,...,...,...,...,...,...,...,...
115,The Borek Bakehouse,4.4,"Bakery, Middle Eastern",1,-37.807711,144.960005,26,QV Market-Elizabeth St (West)
116,Chickcha,3.7,"Coffee and Tea, Cafe Food",2,-37.809073,144.961767,26,QV Market-Elizabeth St (West)
117,Damiao Panda Fan 大妙火锅 · 川菜,3.0,"Hot Pot, Sichuan, Chinese",3,-37.809614,144.961199,26,QV Market-Elizabeth St (West)
118,Mansae Korean BBQ & Bar,4.5,"Korean, Korean BBQ",4,-37.809563,144.959607,26,QV Market-Elizabeth St (West)


In [25]:
# List all unique eatery types for further reduction into broader categories
pd.set_option('display.max_rows', None)

# Confirm total unique values in the Type column, for consolidation
list(z_clean["Type"].sort_values().unique())

['American, Burger',
 'Asian',
 'Asian, Desserts',
 'Asian, Desserts, Ice Cream',
 'Asian, Indonesian',
 'BBQ, Brazilian, Steak',
 'Bakery',
 'Bakery, Coffee and Tea',
 'Bakery, Desserts',
 'Bakery, Middle Eastern',
 'Bar Food',
 'Beverages, Bubble Tea, Tea',
 'Beverages, Healthy Food, Salad',
 'Bubble Tea',
 'Bubble Tea, Asian',
 'Bubble Tea, Taiwanese',
 'Bubble Tea, Tea',
 'Bubble Tea, Tea, Beverages',
 'Burger, Bar Food',
 'Cafe Food',
 'Cafe Food, Coffee and Tea',
 'Cafe Food, Coffee and Tea, Sandwich',
 'Cafe Food, Italian',
 'Cafe Food, Salad, Coffee and Tea',
 'Chinese, BBQ',
 'Chinese, Coffee and Tea',
 'Chinese, Dumplings',
 'Chinese, Dumplings, Hot Pot',
 'Chinese, Hot Pot, Asian, BBQ, Cabinet Food',
 'Chinese, Hot Pot, Sichuan',
 'Chinese, Malatang',
 'Chinese, Malaysian',
 'Chinese, Street Food',
 'Coffee and Tea',
 'Coffee and Tea, Beverages',
 'Coffee and Tea, Cafe Food',
 'Coffee and Tea, Cafe Food, Sandwich',
 'Coffee and Tea, Healthy Food, Salad, Sandwich',
 'Coffee a

In [26]:
# Reduce eateries to the following types: 
# restaurant (regardless of ethnicity), cafe, beverages (including bubble tea, juices, etc.), 
# bar, bakery, desserts, pub, fast food (takeaway),

z_clean_reduced = z_clean.replace({'African, Bar Food': 'Bar',
    'African, Contemporary': 'Restaurant',
    'American, BBQ': 'Restaurant',
    'American, BBQ, Sandwich': 'Cafe',
    'American, Burger': 'Restaurant',
    'American, Cafe Food, Coffee and Tea': 'Cafe',
    'American, Desserts': 'Desserts',
    'American, Grill': 'Restaurant',
    'Asian': 'Restaurant',
    'Asian Fusion, Pan Asian': 'Restaurant',
    'Asian, BBQ, Chinese': 'Restaurant',
    'Asian, Desserts, Ice Cream': 'Restaurant',
    'Asian, Fusion': 'Restaurant',
    'Asian, Indonesian': 'Restaurant',
    'Asian, Japanese': 'Restaurant',
    'Asian, Vietnamese': 'Restaurant',
    'Australian, Contemporary': 'Restaurant',
    'BBQ, American': 'Restaurant',
    'BBQ, Brazilian, Steak': 'Restaurant',
    'Bakery, Coffee and Tea': 'Cafe',
    'Bakery, Coffee and Tea, Sandwich': 'Cafe',
    'Bakery, Desserts': 'Bakery',
    'Bakery, Middle Eastern': 'Bakery',
    'Bakery, Patisserie, Coffee and Tea': 'Bakery',
    'Bakery, Sandwich': 'Bakery',
    'Bar Food': 'Bar',
    'Bar Food, Modern Australian': 'Bar',
    'Beverages, Bubble Tea': 'Beverages',
    'British, Pub Food': 'Pub',
    'Bubble Tea': 'Beverages',
    'Bubble Tea, Asian': 'Beverages',
    'Bubble Tea, Asian, Tea': 'Beverages',
    'Bubble Tea, Beverages': 'Beverages',
    'Bubble Tea, Desserts': 'Beverages',
    'Bubble Tea, Taiwanese': 'Beverages',
    'Bubble Tea, Tea': 'Beverages',
    'Bubble Tea, Tea, Beverages': 'Beverages',
    'Burger': 'Fast Food',
    'Burger, Bar Food': 'Bar',
    'Burger, Desserts': 'Fast Food',
    'Cafe Food': 'Cafe',
    'Cafe Food, Bar Food': 'Cafe',
    'Cafe Food, Coffee and Tea': 'Cafe',
    'Cafe Food, Coffee and Tea, Desserts': 'Cafe',
    'Cafe Food, Coffee and Tea, Sandwich': 'Cafe',
    'Cafe Food, Desserts, Coffee and Tea': 'Cafe',
    'Cafe Food, Italian': 'Cafe',
    'Cafe Food, Salad, Coffee and Tea': 'Cafe',
    'Chinese': 'Restaurant',
    'Chinese, Asian': 'Restaurant',
    'Chinese, Asian, Dumplings': 'Restaurant',
    'Chinese, Asian, Fast Food': 'Restaurant',
    'Chinese, Asian, Yum Cha': 'Restaurant',
    'Chinese, BBQ': 'Restaurant',
    'Chinese, Coffee and Tea': 'Cafe',
    'Chinese, Contemporary, Asian': 'Restaurant',
    'Chinese, Dumplings': 'Restaurant',
    'Chinese, Hot Pot': 'Restaurant',
    'Chinese, Hot Pot, Sichuan': 'Restaurant',
    'Chinese, Malatang': 'Restaurant',
    'Chinese, Malaysian': 'Restaurant',
    'Chinese, Sichuan': 'Restaurant',
    'Chinese, Taiwanese': 'Restaurant',
    'Coffee and Tea': 'Cafe',
    'Coffee and Tea, Bakery, Meat Pie': 'Cafe',
    'Coffee and Tea, Beverages': 'Cafe',
    'Coffee and Tea, Cafe Food': 'Cafe',
    'Coffee and Tea, Cafe Food, Burger': 'Cafe',
    'Coffee and Tea, Desserts': 'Cafe',
    'Coffee and Tea, Pizza, Cafe Food': 'Cafe',
    'Coffee and Tea, Sandwich': 'Cafe',
    'Coffee and Tea, Sandwich, Cafe Food': 'Cafe',
    'Coffee and Tea, Sandwich, Desserts': 'Cafe',
    'Colombian, Cafe Food': 'Cafe',
    'Contemporary, Italian, Australian': 'Restaurant',
    'Continental, Coffee and Tea, Cafe Food': 'Cafe',
    'Desserts, Beverages, Vietnamese': 'Desserts',
    'Desserts, Cafe Food': 'Desserts',
    'Desserts, Coffee and Tea': 'Desserts',
    'Desserts, Coffee and Tea, Cafe Food': 'Desserts',
    'Desserts, French, Coffee and Tea': 'Desserts',
    'Desserts, Frozen Yogurt': 'Desserts',
    'Desserts, Ice Cream': 'Desserts',
    'Drinks Only': 'Beverages',
    'Dumplings, Japanese, Sushi': 'Restaurant',
    'Dutch, Desserts': 'Desserts',
    'European': 'Restaurant',
    'European, Coffee and Tea': 'Cafe',
    'European, Contemporary': 'Restaurant',
    'European, Desserts, Coffee and Tea': 'Cafe',
    'European, Italian': 'Restaurant',
    'Fast Food, Fried Chicken': 'Fast Food',
    'Fast Food, Japanese, Sushi': 'Fast Food',
    'Fast Food, Pizza': 'Fast Food',
    'Fast Food, Taiwanese, Fried Chicken': 'Fast Food',
    'French': 'Restaurant',
    'French, Crepes': 'Restaurant',
    'French, Crepes, Coffee and Tea, Cafe Food': 'Cafe',
    'French, Desserts, Coffee and Tea': 'Cafe',
    'Fried Chicken, Street Food, Korean': 'Restaurant',
    'Greek': 'Restaurant',
    'Greek, Street Food': 'Restaurant',
    'Healthy Food, Asian, Vegetarian': 'Restaurant',
    'Healthy Food, Cafe Food, Coffee and Tea, Sandwich': 'Cafe',
    'Healthy Food, Coffee and Tea, Cafe Food': 'Cafe',
    'Hot Pot, Chinese': 'Restaurant',
    'Hot Pot, Chinese, Asian': 'Restaurant',
    'Ice Cream': 'Desserts',
    'Ice Cream, Cafe Food': 'Desserts',
    'Ice Cream, Desserts': 'Desserts',
    'Indian': 'Restaurant',
    'Indian, Healthy Food, Vegetarian': 'Restaurant',
    'Indian, South Indian': 'Restaurant',
    'Indonesian': 'Restaurant',
    'Italian': 'Restaurant',
    'Italian, Coffee and Tea': 'Cafe',
    'Italian, Coffee and Tea, Cafe Food': 'Cafe',
    'Italian, Pizza': 'Restaurant',
    'Japanese': 'Restaurant',
    'Japanese, Bubble Tea': 'Beverages',
    'Japanese, Cafe Food, Coffee and Tea': 'Cafe',
    'Japanese, Desserts, Coffee and Tea': 'Cafe',
    'Japanese, Fusion': 'Restaurant',
    'Japanese, Fusion, Asian': 'Restaurant',
    'Japanese, Korean': 'Restaurant',
    'Japanese, Ramen': 'Restaurant',
    'Japanese, Sandwich, Coffee and Tea': 'Cafe',
    'Japanese, Sushi': 'Restaurant',
    'Juices, Beverages': 'Beverages',
    'Juices, Coffee and Tea, Cafe Food': 'Beverages',
    'Juices, Healthy Food': 'Beverages',
    'Kebab, Bakery': 'Fast Food',
    'Kebab, Turkish': 'Fast Food',
    'Korean': 'Restaurant',
    'Korean BBQ, Korean': 'Restaurant',
    'Korean BBQ, Korean, Asian': 'Restaurant',
    'Korean, Asian': 'Restaurant',
    'Korean, Burger, Bar Food, American': 'Bar',
    'Korean, Fast Food': 'Fast Food',
    'Korean, Fried Chicken': 'Restaurant',
    'Korean, Fried Chicken, Asian': 'Restaurant',
    'Korean, Korean BBQ': 'Restaurant',
    'Latin American, Mexican': 'Restaurant',
    'Malaysian': 'Restaurant',
    'Mediterranean': 'Restaurant',
    'Mediterranean, Cafe Food, Coffee and Tea': 'Cafe',
    'Mediterranean, Coffee and Tea, Cafe Food': 'Cafe',
    'Mediterranean, Modern Australian': 'Restaurant',
    'Mexican': 'Restaurant',
    'Middle Eastern': 'Restaurant',
    'Modern Australian': 'Restaurant',
    'Modern Australian, Bar Food': 'Bar',
    'Modern Australian, Coffee and Tea, Cafe Food': 'Cafe',
    'Modern Australian, Fusion, Asian': 'Restaurant',
    'Modern Australian, Steak, Grill': 'Restaurant',
    'Modern European': 'Restaurant',
    'Modern European, Bar Food': 'Bar',
    'Modern European, Contemporary': 'Restaurant',
    'Nepalese': 'Restaurant',
    'Pizza': 'Restaurant',
    'Pizza, Fast Food': 'Fast Food',
    'Pizza, Italian': 'Restaurant',
    'Pizza, Kebab, Fast Food': 'Fast Food',
    'Pizza, Pub Food': 'Pub',
    'Pizza, Sandwich': 'Fast Food',
    'Poké': 'Fast Food',
    'Poké, Healthy Food, Fusion': 'Fast Food',
    'Pub Food': 'Pub',
    'Pub Food, Irish': 'Pub',
    'Pub Food, Modern Australian': 'Pub',
    'Salad, Cafe Food': 'Cafe',
    'Sandwich': 'Cafe',
    'Sandwich, Cafe Food': 'Cafe',
    'Sandwich, Coffee and Tea': 'Cafe',
    'Sandwich, Coffee and Tea, Cafe Food': 'Cafe',
    'Sandwich, Drinks Only': 'Cafe',
    'Spanish, Tapas': 'Restaurant',
    'Steak, Modern Australian': 'Restaurant',
    'Sushi, Japanese': 'Restaurant',
    'Thai': 'Restaurant',
    'Thai, Asian': 'Restaurant',
    'Vegetarian, Coffee and Tea, Cafe Food': 'Cafe',
    'Vietnamese': 'Restaurant',
    'Vietnamese, Healthy Food': 'Restaurant',
    'Vietnamese, Indonesian, Malaysian, Asian': 'Restaurant',
    'Vietnamese, Pho': 'Restaurant' })
z_clean_reduced.Type.value_counts()

Restaurant                                       48
Cafe                                             23
Beverages                                        13
Bakery                                            3
Bar                                               3
Desserts                                          2
Fast Food                                         1
Beverages, Healthy Food, Salad                    1
Hot Pot, Sichuan, Chinese                         1
Sandwich, Healthy Food, Coffee and Tea            1
Coffee and Tea, Healthy Food, Salad, Sandwich     1
Coffee and Tea, Cafe Food, Sandwich               1
Malatang                                          1
Healthy Food                                      1
Meat Pie                                          1
Pub                                               1
Contemporary, European, French                    1
Hawaiian, Sushi                                   1
Chinese, Street Food                              1
Sandwich, Fa

In [27]:
# remove dirty data (where Average Ratings = 0 and Price rating = 0)
z_clean_reduced = z_clean_reduced.loc[(z_clean_reduced["Aggregate Rating"] > 0) & (z_clean_reduced["Price"] > 0),:]
z_clean_reduced.count()

Name                105
Aggregate Rating    105
Type                105
Price               105
Latitude            105
Longitude           105
Sensor ID           105
Sensor              105
dtype: int64

In [29]:
list(z_clean_reduced.Type.sort_values().unique())

['Asian, Desserts',
 'Bakery',
 'Bar',
 'Beverages',
 'Beverages, Bubble Tea, Tea',
 'Beverages, Healthy Food, Salad',
 'Cafe',
 'Chinese, Dumplings, Hot Pot',
 'Chinese, Hot Pot, Asian, BBQ, Cabinet Food',
 'Chinese, Street Food',
 'Coffee and Tea, Cafe Food, Sandwich',
 'Coffee and Tea, Healthy Food, Salad, Sandwich',
 'Contemporary, European, French',
 'Desserts',
 'Fast Food',
 'Hawaiian, Sushi',
 'Healthy Food',
 'Hot Pot, Sichuan, Chinese',
 'Japanese, Italian',
 'Malatang',
 'Meat Pie',
 'Pub',
 'Restaurant',
 'Sandwich, Fast Food',
 'Sandwich, Healthy Food, Coffee and Tea']

In [30]:
# final cleanup of the Type column

z_clean_reduced = z_clean_reduced.replace({
    'Asian, Desserts':'Desserts',
    'Beverages, Bubble Tea, Tea':'Beverages',
    'Beverages, Healthy Food, Salad':'Beverages',
    'Chinese, Dumplings, Hot Pot':'Restaurant',
    'Chinese, Hot Pot, Asian, BBQ, Cabinet Food':'Restaurant',
    'Chinese, Street Food':'Restaurant',
    'Coffee and Tea, Cafe Food, Sandwich':'Cafe',
    'Coffee and Tea, Healthy Food, Salad, Sandwich':'Cafe',
    'Contemporary, European, French':'Restaurant',
    'Hawaiian, Sushi':'Restaurant',
    'Healthy Food':'Restaurant',
    'Hot Pot, Sichuan, Chinese':'Restaurant',
    'Japanese, Italian':'Restaurant',
    'Malatang':'Restaurant',
    'Meat Pie':'Bakery',
    'Sandwich, Fast Food':'Fast Food',
    'Sandwich, Healthy Food, Coffee and Tea':'Cafe'})
z_clean_reduced.Type.value_counts()

Restaurant    54
Cafe          25
Beverages     14
Bar            3
Desserts       3
Bakery         3
Fast Food      2
Pub            1
Name: Type, dtype: int64

In [17]:
# Save Zomato DataFrame (Unconsolidated Types) to CSV
z_clean_csv = z_clean.to_csv("../Output/Zomato_Restaurants_All.csv")
# Save Zomato DataFrame (Consolidated Types)
z_clean_types_csv = z_clean_reduced.to_csv("../Output/Zomato_Restaurants_All_Consolidated-Types.csv")